## Retriever And Chain With Langchain

In [1]:
from langchain_community.document_loaders import PyPDFLoader
loader= PyPDFLoader("ekaivakriti_pitch_deck.pdf")
pdf_docs = loader.load()


## Splitting

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
)
chunks_documents = text_splitter.split_documents(pdf_docs)

## Vector Embeddings & Vector Store using Faiss


In [3]:
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

db = FAISS.from_documents(chunks_documents, OpenAIEmbeddings())
db

/var/folders/l5/xr8xrbk17796fzjp9gn8sfbh0000gn/T/ipykernel_38577/2877554414.py:5: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  db = FAISS.from_documents(chunks_documents, OpenAIEmbeddings())


## Quering


In [4]:
query = "contact details of ekaivakriti"
retrieved_results = db.similarity_search(query, k=1)
print(retrieved_results[0].page_content)

    

Ready to Transform Your Business with AI?
Let's start your AI journey today
✉
Get in Touch
hello@ekaivakriti.com
Our team of AI experts is ready to discuss your needs
Contact Us
Your Path to AI Success
Ekaivakriti — Your partner in turning AI's limitless potential into real business impact
   

Initial Consultation
Discuss your business goals and challenges
with our AI experts
1

Solution Proposal
Receive a customized AI strategy tailored to
your specific needs
2

Implementation
Begin your AI transformation journey with
our expert team
3
Made with Genspark


# Retriever & Chain
## Chat prompt Template


In [5]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    """You are a helpful assistant. Think step by step before answering, I will reward you if you provide me the correct and most relevant answer. Use the following pieces of context to answer the question at the end.
    <context>
    {context}
    </context>
    Question: {input}
"""
)


# Opensourse LLM


In [6]:
from langchain_community.llms import Ollama
llm = Ollama(model="llama2", temperature=0.1)
llm


/var/folders/l5/xr8xrbk17796fzjp9gn8sfbh0000gn/T/ipykernel_38577/1542955290.py:2: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama2", temperature=0.1)


Ollama(temperature=0.1)

## Chain

In [7]:
from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain = create_stuff_documents_chain(
    llm=llm,
    prompt=prompt
)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, template='You are a helpful assistant. Think step by step before answering, I will reward you if you provide me the correct and most relevant answer. Use the following pieces of context to answer the question at the end.\n    <context>\n    {context}\n    </context>\n    Question: {input}\n'), additional_kwargs={})])
| Ollama(temperature=0.1)
| StrOutputParser(), kwargs={}, config={'run_name': 'stuff_documents_chain'}, config_factories=[])

In [8]:
"""
Retreveral chain:   This chain retrieves relevant documents from the vector store based on the query and then uses the LLM to generate a response based on those documents.     
"""

from langchain.chains import create_retrieval_chain
retriever = db.as_retriever()
retrieval_chain = create_retrieval_chain(
    retriever, document_chain
)

In [9]:
response = retrieval_chain.invoke({"input": query})

print("\n🔍 Query:", response['input'])
print("\n📬 Answer:\n", response['answer'])

print("\n📄 Context Documents:\n")
for i, doc in enumerate(response['context'], 1):
    meta = doc.metadata
    print(f"--- Document {i} ---")
    print(f"Source: {meta.get('source')}")
    print(f"Page: {meta.get('page_label', meta.get('page'))}")
    content_preview = doc.page_content.strip().split('\n')[:5]  # Preview first 5 lines
    for line in content_preview:
        print(" ", line)
    print()
#response['answer']


🔍 Query: contact details of ekaivakriti

📬 Answer:
 Thank you for providing the context! Based on the information provided, I believe the contact details of Ekaivakriti are:

Email: hello@ekaivakriti.com
Phone Number: Not mentioned in the context.
Address: Not mentioned in the context.

Please let me know if you need any further assistance!

📄 Context Documents:

--- Document 1 ---
Source: ekaivakriti_pitch_deck.pdf
Page: 8
  Ready to Transform Your Business with AI?
  Let's start your AI journey today
  ✉
  Get in Touch
  hello@ekaivakriti.com

--- Document 2 ---
Source: ekaivakriti_pitch_deck.pdf
Page: 7
  Why Choose Ekaivakriti
  Our competitive advantages in the AI solutions landscape
  
  Proven AI Expertise
  Our team brings deep expertise in cutting-

--- Document 3 ---
Source: ekaivakriti_pitch_deck.pdf
Page: 1
  EKekaivakriti
  Your AI Innovation Partner
  Transforming businesses with intelligent agents, automation, and data
  insights for companies across all industries
  